## Test extract links Wikipedia
Looks we have a limit of 10000 records see [Notebook](https://github.com/salgo60/spa2Commons/blob/main/Notebook/SPA%20test.ipynb) - Test 5 

A try to combine that search with birth year
* result just 10000 records found


In [3]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2021-09-19 19:59:41.490914


In [4]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://portrattarkiv.se/endpoints/latest.php"
url= "https://xn--portrttarkiv-kcb.se/endpoints/search.php"
url= "https://portrattarkiv.se/endpoints/search.php"  


In [6]:
from tqdm.notebook import tqdm  
SPAdetail = "https://portrattarkiv.se/details/"
print(url)  

#print("\n",row["search"],row["item"])
limit = 10000
listdf = []
for year in range(1780,1910,50):
    for start in range(0,10000,limit):
        print("\n\nyear", year,"\tfrom: ",start,"\tlimit",limit,"")
        encoded_body = json.dumps({
            "limit": limit,
            "from":  start,
            "year":  year,
            "facts":{"URL":"https://sv.wikipedia.org/wiki"}
        })
    
        r = http.request('POST', url,
            headers={'Content-Type': 'application/json'},
            body=encoded_body)
        print("http status ", r.status)
        if r.status != 200:
            print("Exit ",r.status)
            print(r)
            break
        
    
        data = json.loads(r.data.decode('utf-8'),) 
        urls = []

        print("\nTotal: ",data["hits"]["total"])
        print(data["_shards"])
        print("Hits: ",len(data["hits"]["hits"]))
        df = pd.json_normalize(data["hits"]["hits"])
        listdf.append(df)

https://portrattarkiv.se/endpoints/search.php


year 1780 	from:  0 	limit 10000 
http status  200

Total:  16489
{'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}
Hits:  10000


year 1830 	from:  0 	limit 10000 
http status  200

Total:  16489
{'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}
Hits:  10000


year 1880 	from:  0 	limit 10000 
http status  200

Total:  16489
{'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}
Hits:  10000


In [ ]:
len(listdf)

In [ ]:
dfTot = pd.concat(listdf) 

In [ ]:
#for c in dfTot.columns:
#    print(c)

In [ ]:
len(dfTot)

In [ ]:
 df = dfTot.drop_duplicates(subset = ["_id"])


In [ ]:
len(df[{"_id","_source.Facts.URL"}])

In [ ]:
from wikidata.client import Client
from tqdm.notebook import tqdm 

def checkSPA(wikidata_id):
    # get SPA 
    client = Client()
    entity = client.get(wikidata_id, load=True)
    try:
        spa_prop = entity[client.get('P4819')]
    except:
        return False
    return spa_prop
def getWD (url):
    baseUrl = "https://sv.wikipedia.org/w/api.php?action=query&prop=pageprops|wbentityusage&titles="
    title = url.replace("https://sv.wikipedia.org/wiki/","")
    urlWD = baseUrl + title
    wdr = http.request('Get', urlWD + "&format=json",
                 headers={'Content-Type': 'application/json'})

    if wdr.status != 200:
        print("Error ", wdr.status)
        return False
    
    datawd = json.loads(wdr.data.decode('utf-8'),) 
    pg_dict = datawd['query']['pages']
    pg_key = list(pg_dict.keys())[0]
    wikidata_id = pg_dict[pg_key]['pageprops']['wikibase_item']
    return wikidata_id

for index, row in tqdm( df.iterrows(),total= df.shape[0]):
    #print(row['_id'], row['_source.Facts.URL'])
    for n in row['_source.Facts.URL']:
        if "wikipedia" in n.lower():
            try: 
                wdrec = getWD(n)
                if checkSPA(wdrec) == False:
                    print ("Todo",row['_id'],wdrec,n)
                #print (n,"Value :",getWD(n))
            except:
                pass
  